In [1]:
!pip install -q  requests beautifulsoup4 lxml  langchain-google-genai langgraph langchain google-generativeai google-api-python-client google-auth-httplib2 google-auth-oauthlib google-genai



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 825.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 7.1 MB/s eta 0:00:00


In [10]:
from langchain.prompts import PromptTemplate
import os
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from langchain_google_genai import ChatGoogleGenerativeAI
from google.oauth2 import service_account
from googleapiclient.discovery import build
from urllib.parse import urljoin

In [21]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
# Config
os.environ["GOOGLE_API_KEY"] = ********
OEIL_URL            = "https://oeil.secure.europarl.europa.eu/oeil/en/search"
SERVICE_ACCOUNT_FILE= "/content/drive/MyDrive/gen-lang-client-0687351699-a445369d8c4f.json"
SHEET_ID            = "1jlRfCX1iYsSOrkixGy5mPsuBpKkv-ObAvQfEgrgS_Cs"
TASKLIST_ID         = "@default"


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [12]:
def fetch_html_for_date(date: datetime) -> str:
    ds = date.strftime("%Y%m%d")
    print(f"[DEBUG] fetch_html_for_date → sessionDay.day={ds}")
    r = requests.get(
        OEIL_URL,
        params={
            "sessionDay.allDays": "false",
            "sessionDay.day": ds,
            "sessionDay.type": "ALL"
        },
        headers={"User-Agent": "Mozilla/5.0"}
    )
    print(f"[DEBUG]   Status Code: {r.status_code}")
    html = r.text
    print(f"[DEBUG]   HTML length: {len(html)} caractères")
    print(f"[DEBUG]   HTML preview:\n{html[:20000]!r}\n")

    return html

In [23]:
def parse_procedures(html: str) -> list[dict]:
    """Parse le HTML et retourne la liste des procédures, y compris la description."""
    soup = BeautifulSoup(html, "lxml")
    items = []
    for wrapper in soup.select("div.erpl_document-wrapper"):
        ref_tag     = wrapper.select_one("h3 span.t-item")
        title_tag   = wrapper.select_one("h3.erpl_document-title a.t-y")
        desc_tag    = wrapper.select_one("div.erpl_document-body p")
        comm_tag    = wrapper.select_one("span.erpl_badge-committee")
        rap_tag     = wrapper.select_one("span.erpl_document-subtitle-author")
        pdf_tag     = wrapper.select_one("a.erpl_document-subtitle-pdf")
        date_tag     = wrapper.select_one("div.mt-1 p")


        items.append({
            "reference":   ref_tag.get_text(strip=True) if ref_tag else "",
            "title":       title_tag.get_text(strip=True) if title_tag else "",
            "description": desc_tag.get_text(strip=True) if desc_tag else "",
            "committee":   comm_tag.get_text(strip=True) if comm_tag else "",
            "rapporteur":  rap_tag.get_text(strip=True) if rap_tag else "",
            "pdf":         pdf_tag["href"] if pdf_tag else "",
            "date":        date_tag.get_text(strip=True) if date_tag else "",
        })
    return items

In [24]:
items

[{'reference': '2025/0044(COD)\n                    ***ILegal Affairs',
  'title': '2025/0044(COD)\n                    ***ILegal Affairs',
  'description': 'Certain corporate sustainability reporting and due diligence requirements: application dates',
  'committee': 'Legal Affairs',
  'rapporteur': '',
  'pdf': '/oeil/en/procedure-file/pdf?reference=2025/0044(COD)'},
 {'reference': '2024/2111(BUI)Budgets',
  'title': '2024/2111(BUI)Budgets',
  'description': '2026 budget: estimates of revenue and expenditure, Section I – European Parliament',
  'committee': 'Budgets',
  'rapporteur': 'NEMEC Matjaž (S&D)',
  'pdf': '/oeil/en/procedure-file/pdf?reference=2024/2111(BUI)'},
 {'reference': '2025/2640(RSP)',
  'title': '2025/2640(RSP)',
  'description': 'Health care related tourism: protecting EU patients abroad',
  'committee': '',
  'rapporteur': '',
  'pdf': '/oeil/en/procedure-file/pdf?reference=2025/2640(RSP)'},
 {'reference': '2025/2639(RSP)',
  'title': '2025/2639(RSP)',
  'descripti

In [25]:
def is_sustainable(item: dict) -> bool:
    prompt = f"""
Est-ce que ce document législatif concerne directement la durabilité
(environnement, économie circulaire, pollution…)? Réponds strictement “yes” ou “no”.

Title: {item['title']}
Description: {item['description']}
Committee: {item['committee']}
"""
    response = llm.invoke(prompt)
    content  = getattr(response, "content", response).strip()

    try:
        data   = json.loads(content)
        answer = data.get("answer", "").lower()
    except json.JSONDecodeError:
        answer = content.lower()
    return answer == "yes"


In [47]:
def sheets_service():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/spreadsheets"]
    )
    return build("sheets", "v4", credentials=creds)



BASE_URL = "https://oeil.secure.europarl.europa.eu/"

def append_to_sheet(items: list[dict],hier: datetime):
    srv = sheets_service()

    headers = [["Reference Number", "Committee", "Rapporteur", "Title/Description","Description" ,"PDF Link","Date"]]
    srv.spreadsheets().values().update(
        spreadsheetId=SHEET_ID,
        range="'Feuille 1'!A1:G1",
        valueInputOption="RAW",
        body={"values": headers}
    ).execute()


    values = []
    for it in items:
        pdf_full = urljoin(BASE_URL, it["pdf"])
        values.append([
            it["reference"],
            it["committee"],
            it["rapporteur"],
            it["title"],
            it["description"],
            pdf_full,
            hier.strftime("%d/%m/%Y")

        ])


    srv.spreadsheets().values().append(
        spreadsheetId=SHEET_ID,
        range="'Feuille 1'!A2:E",
        valueInputOption="RAW",
        insertDataOption="INSERT_ROWS",
        body={"values": values}
    ).execute()

In [31]:

def tasks_service():

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/tasks"]
    )
    return build("tasks", "v1", credentials=creds)




def create_tasks(items: list[dict]):
    hier = (datetime.utcnow() - timedelta(days=1)).date()
    srv = tasks_service()
    tomorrow = (datetime.utcnow() + timedelta(days=1)).isoformat() + "Z"
    for it in items:
        pdf_full = urljoin(BASE_URL, it["pdf"])
        srv.tasks().insert(
            tasklist=TASKLIST_ID,
            body={
                "title": f"Étudier {it['reference']}",
                "notes": (
                    f"Titre       : {it['title']}\n"
                    f"Commission  : {it['committee']}\n"
                    f"Rapporteur  : {it['rapporteur']}\n"
                    f"PDF Link    : {pdf_full}"
                    f"Description : {it['description']}"
                    f"Date        : {hier}"
                ),
                "due": tomorrow,
            }
        ).execute()

In [38]:
def main():
    hier      = datetime.utcnow() - timedelta(days=1)
    html      = fetch_html_for_date(hier)
    all_items = parse_procedures(html)
    for it in parse_procedures(html):
      print("—", it["reference"], "|", it["title"])
      print("  Description:", it["description"][:100], "…\n")

    sus = []
    for item in all_items:
        if is_sustainable(item):
            sus.append(item)

    print(f"[DEBUG] main → {len(sus)} procédures retenues après filtre sustainability")
    append_to_sheet(sus,hier)
    create_tasks(sus)
    return sus

if __name__ == "__main__":
    main()

[DEBUG] fetch_html_for_date → sessionDay.day=20250514
[DEBUG]   Status Code: 200
[DEBUG]   HTML length: 60232 caractères
[DEBUG]   HTML preview:
'<!doctype html>\n<html lang="en">\n\n\n<head>\n\n    \n    <title>Search | Legislative Observatory | European Parliament</title>\n\n    <!-- Required meta tags -->\n    <meta charset="utf-8">\n    <meta content="@project.version@" name="project_version"/>\n    <meta content="@build.number@" name="build_number"/>\n    <meta content="@build.time@" name="build_time"/>\n\n    <!-- Commons meta tags -->\n    <meta content="https://www.europarl.europa.eu">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n    <meta content="IE=edge" http-equiv="X-UA-Compatible">\n    <meta content="no-cache" http-equiv="cache-control">\n    <meta content="-1" http-equiv="expires">\n    <meta content="no-cache" http-equiv="pragma">\n    <meta content="text/html; charset=UTF-8" http-equiv="Content-Type">\n    <meta content="

## Test on date 3-04-2025 beacause we are sure that  sustainability procedures exist

In [39]:
from datetime import datetime

date_cible = datetime(2025, 4, 3)

html = fetch_html_for_date(date_cible)

items = parse_procedures(html)

print(f"Procedures pour le {date_cible.strftime('%d/%m/%Y')}:")
for it in items:
    print(f"- {it['reference']} | {it['title']} | {it['committee']} | {it['rapporteur']}| {it['description']}")


[DEBUG] fetch_html_for_date → sessionDay.day=20250403
[DEBUG]   Status Code: 200
[DEBUG]   HTML length: 78088 caractères
[DEBUG]   HTML preview:
'<!doctype html>\n<html lang="en">\n\n\n<head>\n\n    \n    <title>Search | Legislative Observatory | European Parliament</title>\n\n    <!-- Required meta tags -->\n    <meta charset="utf-8">\n    <meta content="@project.version@" name="project_version"/>\n    <meta content="@build.number@" name="build_number"/>\n    <meta content="@build.time@" name="build_time"/>\n\n    <!-- Commons meta tags -->\n    <meta content="https://www.europarl.europa.eu">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n    <meta content="IE=edge" http-equiv="X-UA-Compatible">\n    <meta content="no-cache" http-equiv="cache-control">\n    <meta content="-1" http-equiv="expires">\n    <meta content="no-cache" http-equiv="pragma">\n    <meta content="text/html; charset=UTF-8" http-equiv="Content-Type">\n    <meta content="

In [40]:
 # 2) On applique la classification Gemini
sus_items = []
for item in items:
    if is_sustainable(item):
        sus_items.append(item)

# 3) Affichage simple pour vérifier
print(f"{len(sus_items)} procédures durables détectées :")
for it in sus_items:
    print(f"– {it['reference']} | {it['title']}| {it['description']}")




2 procédures durables détectées :
– 2025/0044(COD)
                    ***ILegal Affairs | 2025/0044(COD)
                    ***ILegal Affairs| Certain corporate sustainability reporting and due diligence requirements: application dates
– 2025/2536(RSP)Industry, Research and Energy | 2025/2536(RSP)Industry, Research and Energy| Resolution on energy-intensive industries


In [48]:
# 4) Stockage dans Google Sheets
append_to_sheet(sus_items,date_cible)

# 5) Création des Google Tasks
create_tasks(sus_items)